## Librerie

In [1]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd

## Parametri problema

In [2]:
# Consumi famiglia 1, famiglia 2, ...
consumi = np.array([1500, 5700, 3400, 300])

# Produzioni ricetta1, ricetta2, ...
produzioni = np.array([3000, 6300, 1200])

# Ricette
#           | Ricetta1 | Ricetta2 | ...
# --------------------------------------
# Famiglia1 |          |          |
# Famiglia2 |          |          |
# ...
ricette = np.array([
    0.25, 0.2, 0.3,
    0.43, 0.5, 0.35,
    0.3, 0.27, 0.35,
    0.02, 0.03,  0
])

# Composizioni ricette per famiglia
#           | Materiale1 | Materiale2 | ...
# --------------------------------------
# Famiglia1 |            |            |
# Famiglia2 |            |            |
# ...
composizioni_famiglia = np.array([
    0.58, 0.42, 0,
    1, 0, 0,
    0, 1, 0,
    0, 0, 1
])

# Range ammissibile percentuale materiale per ricetta
#           | Materiale1        | Materiale2 | ...
# ------------------------------------------------
# Ricetta1  | (val att. ,range) |            |
# Ricetta2  |                   |            |
# ...
range_ric_mat = np.array([
    [(0.58, 0.01), (0.396, 0.003), (0.024, 0.001)], # ricetta 0
    [(0.625, 0.005), (None, None), (None, None)],
    [(0.62, 0.01), (None, None), (None, None)],
])

In [3]:
%run ./importazione.ipynb

(10,)
(28,)
(28, 10)
(28, 13)
(10, 13)
(10, 13)
(10, 13)


In [4]:
print('Verifica dimensioni matrici')
print(f'Consumi (num. famiglie)): {consumi.shape}')
print(f'Produzioni (num. ricette): {produzioni.shape}')
print(f'Ricette (famiglie x ricette): {ricette.shape}')
print(f'Composizioni (famiglie x materiali): {composizioni_famiglia.shape}')
print(f'Range (ricette x materiali): {range_ric_mat.shape}')

Verifica dimensioni matrici
Consumi (num. famiglie)): (28,)
Produzioni (num. ricette): (10,)
Ricette (famiglie x ricette): (280,)
Composizioni (famiglie x materiali): (364,)
Range (ricette x materiali): (10, 13)


## Accrocchio per compattare ricetta

In [5]:
POSITIONS_COMP = []
RICETTA_COMP = []

for index, value in enumerate(ricette):
    if value != 0:
        POSITIONS_COMP.append(index)
        RICETTA_COMP.append(value)

def rebuild_ricetta(ricetta_comp):
    rebuild = np.zeros(len(ricette))
    for index, pos in enumerate(POSITIONS_COMP):
        rebuild[pos] = ricetta_comp[index]
    return rebuild

print(rebuild_ricetta(RICETTA_COMP)[0:20])
print(ricette[0:20])

[0.    0.03  0.    0.    0.    0.    0.    0.    0.    0.    0.46  0.45
 0.05  0.415 0.    0.    0.    0.    0.019 0.   ]
[0.    0.03  0.    0.    0.    0.    0.    0.    0.    0.    0.46  0.45
 0.05  0.415 0.    0.    0.    0.    0.019 0.   ]


## Calcolo resa globale

In [6]:
tot_consumi = np.sum(consumi)
tot_produzioni = np.sum(produzioni)
resa_globale = tot_consumi / tot_produzioni
f'{resa_globale=}'

'resa_globale=np.float64(1.0389346559077537)'

## Funzioni di calcolo

In [7]:
# Calcola matrice consumi moltiplicando matrice ricetta in input per produzioni
def calc_mat_consumi(ricetta):
    ricetta = rebuild_ricetta(ricetta) #REBUILD
    return ricetta.reshape(-1, len(produzioni)) * produzioni

In [8]:
# Calcola vettore consumi complessivi partendo da produzioni iniziali e matrice consumi
def calc_tot_consumi(matrice_consumi):
    return np.sum(matrice_consumi, axis=1)

In [9]:
# Calcolo errore su totali consumi
def calc_err_totali(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_consumi = calc_tot_consumi(matrice_consumi)
    tot_err = np.sum(np.square(tot_consumi-consumi))
    return tot_err

In [10]:
# Calcola rese per famiglia 
# (consumi per famiglia / produzione)
def calc_tot_resa(matrice_consumi):
    return np.sum(matrice_consumi, axis=0)/produzioni

In [31]:
# Calcolo errore su percentuali prod. effettive rispetto a resa totale
def calc_error_resa(ricetta):
    matrice_consumi = calc_mat_consumi(ricetta)
    tot_resa = calc_tot_resa(matrice_consumi)
    return np.sum(np.square(tot_resa - resa_globale))

## Analisi composizione

In [12]:
# Matrice consumi usando ricetta iniziale
# cons_fam = calc_mat_consumi(ricette)
# pd.DataFrame(cons_fam)

In [13]:
# Reshape della matrice composizioni (famiglia vs materiale)
# compos = composizioni_famiglia.reshape(len(consumi), -1)
# pd.DataFrame(compos)

In [14]:
# Devo calcolare quanto materiale per ciascuna ricetta

# Proviamo per prima colonna matrice consumi (quindi prima ricetta)
# Moltiplico la colonna consumi di quella ricetta per le singole colonne della matrice composizione

# cons_ricetta0 = np.vstack(cons_fam[:,0])
# print(cons_ricetta0)
# print('')
# cons_materiali_ricetta0 = cons_ricetta0 * compos
# print(cons_materiali_ricetta0)
# print('')
# tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
# print(tot_cons_ricetta0)
# print('')
# print('Composizione ricetta 0')
# tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0)
# print(tot_perc_ricetta0)

In [15]:
# Percentuali di ciascun materiale in una ricetta

# Verifica dimensioni matrici
# Consumi (num. famiglie)): (28,)
# Produzioni (num. ricette): (10,)
# Ricette (famiglie x ricette): (280,)
# Composizioni (famiglie x materiali): (364,)
# Range (ricette x materiali): (28, 13)

def perc_mat(ricetta, id_ric):
    cons_fam = calc_mat_consumi(ricetta)
    compos = composizioni_famiglia.reshape(len(consumi), -1)
    cons_ricetta0 = np.vstack(cons_fam[:,id_ric])
    cons_materiali_ricetta0 = cons_ricetta0 * compos
    tot_cons_ricetta0 = np.sum(cons_materiali_ricetta0, axis=0)
    tot_perc_ricetta0 = tot_cons_ricetta0 / np.sum(tot_cons_ricetta0) if np.sum(tot_cons_ricetta0) != 0 else np.zeros((len(tot_cons_ricetta0),1))
    return tot_perc_ricetta0

In [16]:
# Funzione errore percentuale materiale (obiettivo: >=0)
def err_perc_mat(ricetta, id_ric, id_mat, expected_val, expected_error):
    return expected_error - np.abs(perc_mat(ricetta, id_ric=id_ric)[id_mat] - expected_val)

## Ottimizzazione

In [46]:
%%time

constraints = [
    {'type': 'eq', 'fun': calc_err_totali},
]

for id_ric, ric in enumerate(range_ric_mat):
    for id_mat, mat in enumerate(ric):
        if not np.isnan(mat[0]):
            constr = {'type': 'ineq', 'fun': err_perc_mat, 'args': (id_ric, id_mat, mat[0], mat[1])}
            constraints.append(constr)

# constraints = (
#     {'type': 'eq', 'fun': calc_err_totali},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 0, 0.58, 0.01)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 1, 0.396, 0.003)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (0, 2, 0.024, 0.001)},
#     {'type': 'ineq', 'fun': err_perc_mat, 'args': (1, 0, 0.625, 0.005)},
#     # {'type': 'ineq', 'fun': err_perc_mat, 'args': (2, 0, 0.62, 0.01)},
# )

#DEBUG
print(len(constraints))
constraints = constraints[0:60] #DEBUG 40 ok, oltre boom
# del constraints[26]
# del constraints[14]
# del constraints[13]

bounds = list(( (0, 1) for x in range(len(RICETTA_COMP) )))

# Forza a zero i valori che nella ricetta originale sono già nulli
# for index, value in enumerate(ricette):
#     if value == 0:
#         bounds[index] = (0,0)

res = minimize(
    calc_error_resa, 
    #ricette, 
    RICETTA_COMP,
    # method='trust-constr', 
    method='SLSQP',
    constraints=constraints,
    bounds=bounds,
    options={'disp': True, 'maxiter':100}
)

print(res)

# res = minimize(
#     calc_error_resa, 
#     res.x, 
#     method='trust-constr', 
#     # method='SLSQP',
#     constraints=constraints,
#     bounds=bounds,
#     options={'disp': True, 'maxiter':20}
# )

131
Iteration limit reached    (Exit mode 9)
            Current function value: 0.3739559412659561
            Iterations: 100
            Function evaluations: 7894
            Gradient evaluations: 100
 message: Iteration limit reached
 success: False
  status: 9
     fun: 0.3739559412659561
       x: [ 3.582e-02  5.605e-01 ...  1.079e-02  3.842e-03]
     nit: 100
     jac: [ 7.615e-01  3.434e-01 ... -3.282e-02  6.649e-03]
    nfev: 7894
    njev: 100
CPU times: total: 40.8 s
Wall time: 1min 12s


In [47]:
# Esperimento
# Proviamo a escludere un constraint alla volta e vediamo se troviamo una situazione
# in cui il problema converge.

# for i in range(1, len(constraints)):
    
#     print(i)
    
#     const_red = constraints[0:i] + constraints[i+1:]

#     res = minimize(
#         calc_error_resa, 
#         ricette, 
#         method='SLSQP',
#         constraints=const_red,
#         bounds=bounds,
#         options={'disp': False, 'maxiter':100}
#     )

#     print(f'{res.success} - {res.message} ({res.nit} iterations)')


## Verifiche

In [48]:
pd.options.display.float_format = '{:.2f}'.format

In [49]:
result = rebuild_ricetta(res.x)

In [50]:
print("Percentuali aggiustate (in %)")
pd.DataFrame(
    result.reshape(len(consumi), len(produzioni))*100
)

Percentuali aggiustate (in %)


,0,1,2,3,4,5,6,7,8,9
0,0.00,3.58,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,56.05,59.52,17.00,51.75,0.00,0.00,0.00,0.00,19.90,0.00
2,0.00,0.00,42.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,7.55,0.00,0.00,0.00,26.46,0.00
4,0.00,0.00,0.00,0.00,0.00,5.89,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,45.58,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,11.36
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,7.10,0.00
8,41.86,40.39,14.08,24.88,0.00,0.00,0.00,0.00,14.31,0.00
9,0.00,0.00,25.16,0.00,3.12,29.47,0.00,0.00,0.00,0.00


In [51]:
print("Matrice consumi")
pd.DataFrame(
    calc_mat_consumi(res.x)
)

Matrice consumi


,0,1,2,3,4,5,6,7,8,9
0,0.00,1253478.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,3250073.89,20828273.93,1271793.04,48254184.58,0.00,0.00,0.00,0.00,997725.11,0.00
2,0.00,0.00,3155183.77,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,283278.31,0.00,0.00,0.00,1326345.91,0.00
4,0.00,0.00,0.00,0.00,0.00,79978.05,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,715654.40,0.00,0.00
6,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,32481.40
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,356027.42,0.00
8,2427172.49,14132313.29,1053362.76,23204022.03,0.00,0.00,0.00,0.00,717549.93,0.00
9,0.00,0.00,1881837.17,0.00,117208.56,400366.51,0.00,0.00,0.00,0.00


In [52]:
print('Verifica totale consumi')
pd.DataFrame(
    [calc_tot_consumi(calc_mat_consumi(res.x))]
)

Verifica totale consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,1253478.00,74602050.55,3155183.77,1609624.22,79978.05,715654.40,32481.40,356027.42,41534420.50,2399412.24,...,10411199.78,511691.20,59880.12,14271.01,15382.03,33777.30,414022.75,13651.74,130884.26,20688.27


In [53]:
print('Verifica errore consumi')
pd.DataFrame([calc_tot_consumi(calc_mat_consumi(res.x)) - consumi])

Verifica errore consumi


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,6096.00,10762.83,7255.89,1977.62,9714.55,-2622.90,8854.40,19807.42,361.16,4523.24,...,992.78,-974535.80,1998.12,11921.01,10940.03,21572.30,389695.75,13438.74,124488.76,9845.27


In [54]:
print(f'Verifica rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x))]
)

Verifica rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,1.21,1.42,1.04,0.87,1.43,1.02,0.99,1.04,0.98,1.14


In [55]:
print(f'Verifica errore rese (resa globale: {resa_globale:.2f})')
pd.DataFrame(
    [calc_tot_resa(calc_mat_consumi(res.x)) - resa_globale]
)

Verifica errore rese (resa globale: 1.04)


,0,1,2,3,4,5,6,7,8,9
0,0.17,0.38,0.00,-0.17,0.39,-0.02,-0.04,0.00,-0.06,0.10


In [56]:
print('Percentuali materiali ricetta 0')
print(pd.DataFrame(range_ric_mat[0])[['f0']])
print(pd.DataFrame([perc_mat(RICETTA_COMP,0) * 100]))
pd.DataFrame([perc_mat(res.x, 0) * 100])

Percentuali materiali ricetta 0
     f0
0  0.59
1  0.02
2  0.00
3  0.00
4  0.00
5  0.00
6  0.00
7  0.00
8  0.00
9  0.00
10 0.00
11 0.00
12 0.38
     0     1    2    3    4    5    6    7    8    9    10   11    12
0 57.10 11.45 0.13 0.14 0.05 0.00 0.00 0.00 0.00 0.00 0.00 0.00 31.13


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,59.55,3.04,0.13,0.15,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,37.08


In [57]:
print('Percentuali materiali ricetta 1')
print(range_ric_mat[1])
pd.DataFrame([perc_mat(res.x, 1) * 100])

Percentuali materiali ricetta 1
[(5.80000e-01, 0.01) (2.90000e-02, 0.01) (1.65000e-03, 0.01)
 (1.65000e-03, 0.01) (1.00000e-03, 0.01) (1.50000e-04, 0.01)
 (1.00000e-04, 0.01) (5.00000e-05, 0.01) (1.00000e-04, 0.01)
 (7.50000e-05, 0.01) (5.00000e-05, 0.01) (0.00000e+00, 0.01)
 (3.86175e-01, 0.01)]


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,62.70,3.85,0.13,0.14,0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,33.12


In [58]:
print('Percentuali materiali ricetta 2')
print(range_ric_mat[2])
pd.DataFrame([perc_mat(res.x, 2) * 100])

Percentuali materiali ricetta 2
[(6.18000e-01, 0.01) (2.00000e-02, 0.01) (6.50000e-04, 0.01)
 (6.50000e-04, 0.01) (5.00000e-04, 0.01) (1.50000e-04, 0.01)
 (1.00000e-04, 0.01) (5.00000e-05, 0.01) (1.00000e-03, 0.01)
 (7.50000e-05, 0.01) (5.00000e-05, 0.01) (0.00000e+00, 0.01)
 (3.58775e-01, 0.01)]


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,61.63,1.30,0.11,0.11,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,36.82


In [59]:
for index,c in enumerate(constraints):
    # print(c)
    if 'args' in c:
        print(f"{index} - {c['fun'](res.x, *c['args'])} (ric:{c['args'][0]} mat:{c['args'][1]})")
    else:
        print(c['fun'](res.x))

1121057142495.0645
1 - 0.008542213850649683 (ric:0 mat:0)
2 - 7.668971006955655e-05 (ric:0 mat:1)
3 - 0.0096842652839043 (ric:0 mat:2)
4 - 0.009997812857235921 (ric:0 mat:3)
5 - 0.009974158672460865 (ric:0 mat:4)
6 - 0.009849999769357964 (ric:0 mat:5)
7 - 0.009899999779008795 (ric:0 mat:6)
8 - 0.009949999777745688 (ric:0 mat:7)
9 - 0.009899999779008795 (ric:0 mat:8)
10 - 0.009924999772920273 (ric:0 mat:9)
11 - 0.009949999777745688 (ric:0 mat:10)
12 - 0.009999999776482582 (ric:0 mat:11)
13 - 0.0004617303831850744 (ric:1 mat:1)
14 - 0.0096337286401826 (ric:1 mat:2)
15 - 0.009767103271434076 (ric:1 mat:3)
16 - 0.009507276071679074 (ric:1 mat:4)
17 - 0.009875478889626362 (ric:1 mat:5)
18 - 0.00990277056525193 (ric:1 mat:6)
19 - 0.009952770563988824 (ric:1 mat:7)
20 - 0.00990277056525193 (ric:1 mat:8)
21 - 0.009927770559163409 (ric:1 mat:9)
22 - 0.009952770563988824 (ric:1 mat:10)
23 - 0.009999999776482582 (ric:1 mat:11)
24 - 0.008255186402440495 (ric:2 mat:0)
25 - 0.002953009266614806 (ric